**FLIGHT DATA**

In [1]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import dask.dataframe as dd
import seaborn as sns

In [4]:
def segregate_flight_data(source, target):

    if not os.path.exists(target):
        os.makedirs(target)
    
    # Get the list of month folders in the year folder
    month_folders = [f for f in os.listdir(source) if os.path.isdir(os.path.join(source, f))]
    
    # Iterate over each month folder
    for month_folder in month_folders:
        # Construct the source path for the month folder
        month_source_path = os.path.join(source, month_folder)
        
        # Get the list of CSV files in the month folder
        csv_files = [f for f in os.listdir(month_source_path) if f.endswith('.csv')]
        
        # Move each CSV file to the target folder
        for csv_file in csv_files:
            source_file_path = os.path.join(month_source_path, csv_file)
            target_file_path = os.path.join(target, csv_file)
            shutil.move(source_file_path, target_file_path)

In [3]:
target_path = 'flight'
source_2016 = '2016'
source_2017 = '2017'

In [4]:
segregate_flight_data(source_2016, target_path)

NameError: name 'segregate_flight_data' is not defined

In [5]:
segregate_flight_data(source_2017, target_path)

NameError: name 'segregate_flight_data' is not defined

In [33]:
main_dir = 'flight'  
all_csv_files = []

# Gather all .csv files from the main directory
for file in os.listdir(main_dir):
    file_path = os.path.join(main_dir, file)
    if file_path.endswith('.csv'):
        all_csv_files.append(file_path)

# Initialize an empty DataFrame
combined_data = pd.DataFrame()

# Specify required columns
req_col = ['FlightDate', 'Quarter', 'Year', 'Month', 'DayofMonth', 'DepTime', 'DepDel15', 'CRSDepTime', 'DepDelayMinutes', 'OriginAirportID', 'DestAirportID', 'ArrTime', 'CRSArrTime', 'ArrDel15', 'ArrDelayMinutes', 'Origin', 'Dest']

# Read and concatenate all CSV files
for idx, file in enumerate(all_csv_files):
    print(f"Reading file {idx+1}/{len(all_csv_files)}: {file}")
    try:
        data = pd.read_csv(file, usecols=req_col, low_memory=False)
        combined_data = pd.concat([combined_data, data], ignore_index=True)
    except Exception as e:
        print(f"Error reading {file}: {e}")

# List of target airports
airports = ["ATL", "CLT", "DEN", "DFW", "EWR", "IAH", "JFK", "LAS", "LAX", "MCO", "MIA", "ORD", "PHX", "SEA", "SFO"]

# Filter data for the specified airports
combined_data = combined_data[(combined_data['Origin'].isin(airports)) & (combined_data['Dest'].isin(airports))]

# Save the combined data to a CSV file
combined_data.to_csv('Flight_data.csv', index=False, encoding='utf-8')
print("Combined data saved to 'Flight_data.csv'.")


Reading file 1/24: flight/On_Time_On_Time_Performance_2016_8.csv
Reading file 2/24: flight/On_Time_On_Time_Performance_2016_9.csv
Reading file 3/24: flight/On_Time_On_Time_Performance_2017_8.csv
Reading file 4/24: flight/On_Time_On_Time_Performance_2017_9.csv
Reading file 5/24: flight/On_Time_On_Time_Performance_2016_12.csv
Reading file 6/24: flight/On_Time_On_Time_Performance_2016_10.csv
Reading file 7/24: flight/On_Time_On_Time_Performance_2016_11.csv
Reading file 8/24: flight/On_Time_On_Time_Performance_2017_2.csv
Reading file 9/24: flight/On_Time_On_Time_Performance_2017_3.csv
Reading file 10/24: flight/On_Time_On_Time_Performance_2017_1.csv
Reading file 11/24: flight/On_Time_On_Time_Performance_2017_4.csv
Reading file 12/24: flight/On_Time_On_Time_Performance_2017_5.csv
Reading file 13/24: flight/On_Time_On_Time_Performance_2017_7.csv
Reading file 14/24: flight/On_Time_On_Time_Performance_2017_6.csv
Reading file 15/24: flight/On_Time_On_Time_Performance_2016_1.csv
Reading file 16/

In [34]:
combined_flight_data = pd.read_csv("Flight_data.csv")

In [35]:
combined_flight_data.shape

(1877296, 17)

In [36]:
combined_flight_data.nunique()

Year                  2
Quarter               4
Month                12
DayofMonth           31
FlightDate          731
OriginAirportID      15
Origin               15
DestAirportID        15
Dest                 15
CRSDepTime         1255
DepTime            1432
DepDelayMinutes    1082
DepDel15              2
CRSArrTime         1421
ArrTime            1440
ArrDelayMinutes    1075
ArrDel15              2
dtype: int64

In [37]:
combined_flight_data.isnull().sum()

Year                   0
Quarter                0
Month                  0
DayofMonth             0
FlightDate             0
OriginAirportID        0
Origin                 0
DestAirportID          0
Dest                   0
CRSDepTime             0
DepTime            20980
DepDelayMinutes    20987
DepDel15           20987
CRSArrTime             0
ArrTime            22080
ArrDelayMinutes    25860
ArrDel15           25860
dtype: int64

In [38]:
columns_with_null = ['DepTime', 'ArrDelayMinutes', 'ArrDel15', 'DepDel15', 'DepDelayMinutes', 'ArrTime']

In [40]:
cleaned_flight_data = combined_flight_data.dropna(subset = columns_with_null)

In [41]:
cleaned_flight_data.shape

(1851436, 17)

In [42]:
cleaned_flight_data.isnull().sum()

Year               0
Quarter            0
Month              0
DayofMonth         0
FlightDate         0
OriginAirportID    0
Origin             0
DestAirportID      0
Dest               0
CRSDepTime         0
DepTime            0
DepDelayMinutes    0
DepDel15           0
CRSArrTime         0
ArrTime            0
ArrDelayMinutes    0
ArrDel15           0
dtype: int64

In [43]:
df1 = cleaned_flight_data

In [46]:
val = []
for i in df1['CRSDepTime'] : 
    rem = i % 100 
    if rem < 30 : 
        val.append(i - rem)
    else : 
        val.append(i + (100 - rem))
df1['rounded_CRSDepTime'] = val
df1['rounded_CRSDepTime']

/var/folders/x3/6tds_w9j0lb0pr8y5qsz97tm0000gn/T/ipykernel_38909/2521633794.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rounded_CRSDepTime'] = val


0           600
1           800
2          2200
3           100
4          2100
           ... 
1877291    1200
1877292    1400
1877293    1600
1877294    1800
1877295    2200
Name: rounded_CRSDepTime, Length: 1851436, dtype: int64

In [150]:
df1['rounded_CRSDepTime'] = df.apply(lambda row: roundOffTime(row['CRSDepTime'] // 100, row['CRSDepTime'] % 100), axis=1)

/var/folders/x3/6tds_w9j0lb0pr8y5qsz97tm0000gn/T/ipykernel_24755/562525719.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rounded_CRSDepTime'] = df.apply(lambda row: roundOffTime(row['CRSDepTime'] // 100, row['CRSDepTime'] % 100), axis=1)


In [47]:
df1.tail()

,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,Time,rounded_CRSDepTime
1877291,2016,2,5,20,2016-05-20,14747,SEA,11292,DEN,1130,1136.0,6.0,0.0,1515,1523.0,8.0,0.0,1200,1200
1877292,2016,2,5,20,2016-05-20,10397,ATL,14107,PHX,1350,1342.0,0.0,0.0,1505,1443.0,0.0,0.0,1400,1400
1877293,2016,2,5,20,2016-05-20,14107,PHX,10397,ATL,1550,1540.0,0.0,0.0,2236,2213.0,0.0,0.0,1600,1600
1877294,2016,2,5,20,2016-05-20,11292,DEN,12266,IAH,1734,1728.0,0.0,0.0,2110,2035.0,0.0,0.0,1800,1800
1877295,2016,2,5,20,2016-05-20,12266,IAH,12889,LAS,2155,2149.0,0.0,0.0,2328,2306.0,0.0,0.0,2200,2200


**WEATHER DATA**

In [48]:
reqd_columns = ['WindSpeedKmph', 'WindDirDegree', 'WeatherCode', 'precipMM', 'Visibilty', 'Pressure', 'Cloudcover', 'DewPointF',
                    'WindGustKmph', 'tempF', 'WindChillF', 'Humidity', 'date', 'time', 'airport']

In [49]:
def read_and_flatten_json(file_path, airport_name):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    weather_data = data['data']['weather']
    flattened_data = []
    reqd_columns = ['WindSpeedKmph', 'WindDirDegree', 'WeatherCode', 'precipMM', 'Visibilty', 'Pressure', 'Cloudcover', 'DewPointF',
                    'WindGustKmph', 'tempF', 'WindChillF', 'Humidity', 'date', 'time', 'airport']

    for day in weather_data:
        date = day['date']
        hourly_data = day['hourly']
        for hour in hourly_data:
            hour_data = {
                'WindSpeedKmph': hour.get('windspeedKmph', None),
                'WindDirDegree': hour.get('winddirDegree', None),
                'WeatherCode': hour.get('weatherCode', None),
                'precipMM': hour.get('precipMM', None),
                'Visibilty': hour.get('visibility', None),
                'Pressure': hour.get('pressure', None),
                'Cloudcover': hour.get('cloudcover', None),
                'DewPointF': hour.get('DewPointF', None),
                'WindGustKmph': hour.get('WindGustKmph', None),
                'tempF': hour.get('tempF', None),
                'WindChillF': hour.get('WindChillF', None),
                'Humidity': hour.get('humidity', None),
                'date': date,
                'time': hour.get('time', None),
                'airport': airport_name
            }
            flattened_data.append(hour_data)
    
    return pd.DataFrame(flattened_data)

In [50]:
def combine_weather_data(root_path):
    dfs = []  # List to store individual DataFrames

    # Loop through each subfolder
    for subfolder in os.listdir(root_path):
        airport_name = subfolder #saving folder name here so we can assign it to the column
        subfolder_path = os.path.join(root_path, subfolder)

        if os.path.isdir(subfolder_path):
            print(f"Processing airport: {airport_name}")
            # Loop through each JSON file in the subfolder
            for file in os.listdir(subfolder_path):
                if (file.startswith("2016") or file.startswith("2017")) and file.endswith('.json'):  # Process only JSON files
                    file_path = os.path.join(subfolder_path, file)
                    print(f"Processing file: {file_path}")
                    try:
                        # Read and flatten the JSON file
                        df = read_and_flatten_json(file_path, airport_name)
                        # Append the DataFrame to the list
                        dfs.append(df)
                    except Exception as e:
                        print(f"Error reading {file_path}: {e}")

    # Combine all DataFrames
    combined_df = pd.concat(dfs, ignore_index=True)

    # Ensure all DataFrames have the same columns
    for col in reqd_columns:
        if col not in combined_df.columns:
            combined_df[col] = None

    return combined_df


In [51]:
combined_weather_data = combine_weather_data('weather')

Processing airport: IAH
Processing file: weather/IAH/2016-2.json
Processing file: weather/IAH/2017-6.json
Processing file: weather/IAH/2017-7.json
Processing file: weather/IAH/2016-3.json
Processing file: weather/IAH/2016-11.json
Processing file: weather/IAH/2016-8.json
Processing file: weather/IAH/2017-10.json
Processing file: weather/IAH/2016-4.json
Processing file: weather/IAH/2017-1.json
Processing file: weather/IAH/2016-5.json
Processing file: weather/IAH/2017-11.json
Processing file: weather/IAH/2016-9.json
Processing file: weather/IAH/2016-10.json
Processing file: weather/IAH/2017-12.json
Processing file: weather/IAH/2017-2.json
Processing file: weather/IAH/2016-6.json
Processing file: weather/IAH/2016-7.json
Processing file: weather/IAH/2017-3.json
Processing file: weather/IAH/2016-12.json
Processing file: weather/IAH/2017-4.json
Processing file: weather/IAH/2017-8.json
Processing file: weather/IAH/2017-9.json
Processing file: weather/IAH/2016-1.json
Processing file: weather/IA

In [52]:
num_rows = combined_weather_data.shape[0]
num_columns = combined_weather_data.shape[1]

print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

Number of rows: 263160
Number of columns: 15


In [53]:
combined_weather_data.nunique()

WindSpeedKmph     89
WindDirDegree    361
WeatherCode       43
precipMM         281
Visibilty         21
Pressure          72
Cloudcover       101
DewPointF         99
WindGustKmph     122
tempF            131
WindChillF       146
Humidity          98
date             731
time              24
airport           15
dtype: int64

In [54]:
combined_weather_data.isnull().sum()

WindSpeedKmph    0
WindDirDegree    0
WeatherCode      0
precipMM         0
Visibilty        0
Pressure         0
Cloudcover       0
DewPointF        0
WindGustKmph     0
tempF            0
WindChillF       0
Humidity         0
date             0
time             0
airport          0
dtype: int64

In [55]:
combined_weather_data.head()

,WindSpeedKmph,WindDirDegree,WeatherCode,precipMM,Visibilty,Pressure,Cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,Humidity,date,time,airport
0,10,209,122,0.0,10,1012,99,60,14,63,63,90,2016-02-01,0,IAH
1,8,209,122,0.0,10,1012,99,60,11,63,63,90,2016-02-01,100,IAH
2,6,209,122,0.0,10,1012,100,60,9,63,63,90,2016-02-01,200,IAH
3,5,209,122,0.1,10,1012,100,60,6,63,63,90,2016-02-01,300,IAH
4,5,195,122,0.1,10,1012,100,60,6,62,62,90,2016-02-01,400,IAH


In [56]:
combined_weather_data.shape

(263160, 15)

In [57]:
def check_all_dates_present(df, year):
    all_dates = pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31', freq='D')
    
    df_dates = pd.to_datetime(df['date'])
    
    df_dates = df_dates[df_dates.dt.year == year]
    
    missing_dates = set(all_dates) - set(df_dates)
    
    return missing_dates

In [58]:
missing_dates_2016 = check_all_dates_present(combined_weather_data, 2016)

In [59]:
missing_dates_2016

set()

In [60]:
missing_dates_2017 = check_all_dates_present(combined_weather_data, 2017)

In [61]:
missing_dates_2017

set()

In [62]:
df2 = combined_weather_data

In [78]:
df2.to_csv("Weather_data.csv", index = False)

**COMBINING WEATHER AND FLIGHT DATA**

In [63]:
df1

,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,Time,rounded_CRSDepTime
0,2016,3,8,27,2016-08-27,11292,DEN,12889,LAS,600,557.0,0.0,0.0,656,650.0,0.0,0.0,600,600
1,2016,3,8,27,2016-08-27,12889,LAS,13204,MCO,736,733.0,0.0,0.0,1508,1500.0,0.0,0.0,800,800
2,2016,3,8,27,2016-08-27,13204,MCO,14771,SFO,2200,2205.0,5.0,0.0,43,43.0,0.0,0.0,2200,2200
3,2016,3,8,27,2016-08-27,12892,LAX,13204,MCO,105,48.0,0.0,0.0,908,853.0,0.0,0.0,100,100
4,2016,3,8,27,2016-08-27,13204,MCO,12892,LAX,2120,2144.0,24.0,1.0,2335,2335.0,0.0,0.0,2100,2100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877291,2016,2,5,20,2016-05-20,14747,SEA,11292,DEN,1130,1136.0,6.0,0.0,1515,1523.0,8.0,0.0,1200,1200
1877292,2016,2,5,20,2016-05-20,10397,ATL,14107,PHX,1350,1342.0,0.0,0.0,1505,1443.0,0.0,0.0,1400,1400
1877293,2016,2,5,20,2016-05-20,14107,PHX,10397,ATL,1550,1540.0,0.0,0.0,2236,2213.0,0.0,0.0,1600,1600
1877294,2016,2,5,20,2016-05-20,11292,DEN,12266,IAH,1734,1728.0,0.0,0.0,2110,2035.0,0.0,0.0,1800,1800


In [64]:
df2

,WindSpeedKmph,WindDirDegree,WeatherCode,precipMM,Visibilty,Pressure,Cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,Humidity,date,time,airport
0,10,209,122,0.0,10,1012,99,60,14,63,63,90,2016-02-01,0,IAH
1,8,209,122,0.0,10,1012,99,60,11,63,63,90,2016-02-01,100,IAH
2,6,209,122,0.0,10,1012,100,60,9,63,63,90,2016-02-01,200,IAH
3,5,209,122,0.1,10,1012,100,60,6,63,63,90,2016-02-01,300,IAH
4,5,195,122,0.1,10,1012,100,60,6,62,62,90,2016-02-01,400,IAH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263155,19,110,113,0.0,10,1018,0,74,22,87,87,65,2017-05-31,1900,MIA
263156,18,110,113,0.0,10,1018,0,74,21,86,86,67,2017-05-31,2000,MIA
263157,17,110,113,0.0,10,1018,0,74,20,85,85,69,2017-05-31,2100,MIA
263158,15,113,113,0.0,10,1018,8,73,19,84,84,70,2017-05-31,2200,MIA


In [65]:
df1['FlightDate'] = pd.to_datetime(df1['FlightDate'])
df2['date'] = pd.to_datetime(df2['date'])

/var/folders/x3/6tds_w9j0lb0pr8y5qsz97tm0000gn/T/ipykernel_38909/3712761900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['FlightDate'] = pd.to_datetime(df1['FlightDate'])


In [66]:
df1['rounded_CRSDepTime'] = df1['rounded_CRSDepTime'].astype(str)

/var/folders/x3/6tds_w9j0lb0pr8y5qsz97tm0000gn/T/ipykernel_38909/4085270143.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rounded_CRSDepTime'] = df1['rounded_CRSDepTime'].astype(str)


In [67]:
df = df1.merge(df2, how='left', left_on=['FlightDate', 'rounded_CRSDepTime', 'Origin'], right_on=['date', 'time', 'airport'])

In [84]:
df.shape

(1851436, 33)

In [74]:
df = df.drop(['Time'], axis = 1)

In [85]:
df = df.dropna()

In [86]:
df.shape

(1832252, 33)

In [87]:
df.to_csv("merged_data.csv", index = False)

**END OF PRE PROCESSING**